In [1]:
import numpy as np
import galois
from itertools import chain, combinations
GF = galois.GF(2)

In [364]:
# configuration model
# https://en.wikipedia.org/wiki/Configuration_model

n = 16
deg_c = 4 # w_r. Every check has this many bits in it
deg_v = 3 # w_c. Every bit is in this many checks
num_checks = (n*deg_v)//deg_c
k = n - num_checks

vs = np.array([[j for i in range(deg_v)] for j in range(n)]).flatten()
cs = np.array([[j for i in range(deg_c)] for j in range(num_checks)]).flatten()

H = np.zeros((num_checks, n), np.int64)

while (vs.size and cs.size):
    # choose random 'stub' from each array
    double_edge = True
    while(double_edge):
        v_ind = np.random.randint(0, len(vs))
        c_ind = np.random.randint(0, len(cs))

        if (H[cs[c_ind]][vs[v_ind]] != 1):
            double_edge = False
            H[cs[c_ind]][vs[v_ind]] = 1
            vs = np.delete(vs, v_ind)
            cs =np.delete(cs, c_ind)

In [401]:
H = np.loadtxt('./good_ldpc_codes/16_4_3.txt', dtype=np.int8)

In [402]:
hx1 = np.kron(H, np.eye(len(H[0]), dtype=np.int8))
hx2 = np.kron(np.eye(len(H), dtype=np.int8), H.T)
Hx = GF(np.hstack([hx1, hx2]))

hz1 = np.kron(np.eye(len(H[0]), dtype=np.int8), H)
hz2 = np.kron(H.T, np.eye(len(H), dtype=np.int8))
Hz = GF(np.hstack([hz1, hz2]))

In [408]:
np.where(Hx[0])

(array([ 16, 112, 128, 192, 261, 263, 266], dtype=int64),)

In [404]:
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def syn_from_F(F, H):
    eF = GF(np.zeros(H.shape[1], dtype=np.int8))
    np.put(eF, F, [1])
    return set(np.where(H @ eF)[0])

Fx = [list(powerset(np.where(Hx[i])[0]))[1:] for i in range(Hx.shape[0])]
Fx = list(chain(*Fx)) # can take set of this list for slight reduction in size
# Fx = [set(g) for g in Fx] 
Fz = [list(powerset(np.where(Hz[i])[0]))[1:] for i in range(Hz.shape[0])]
Fz = list(chain(*Fz))
# Fz = [set(g) for g in Fz]

sigma_Fx = [syn_from_F(g, Hz) for g in Fx] # set of indices where syndrome is 1
sigma_Fz = [syn_from_F(g, Hx) for g in Fz]

In [409]:
%timeit syn_from_F(Fx[0], Hz)

338 µs ± 52.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [393]:
def ssf(syn, x_z):
    # given a syndrome, syn: sigma_x or sigma_z 
    # x_z false for x stabilizers, true for z stabilizers
    s = set(np.where(syn.copy())[0])
    e = set()
    F = Fx if x_z else Fz
    sigma_F = sigma_Fx if x_z else sigma_Fz
    
    while True:
        max = -1
        max_gen = None
        max_sigma_gen = None
        for g, sigma_g in zip(F, sigma_F):
            s_i = s ^ sigma_g
            if (len(s_i) < len(s)):
                rel_weight = (len(s) - len(s_i)) / len(g)
                if (rel_weight > max):
                    max = rel_weight
                    max_gen = g
                    max_sigma_gen = sigma_g
            else:
                continue

        if (max == -1):
            if (len(s) == 0):
                return e
            else:
                return "FAIL"
        else:
            e = e ^ set(max_gen)
            s = s ^ max_sigma_gen

In [384]:
p = 0.01

# how is this able to decode an error with more than (d-1)/2 errors?

for i in range(1):
    eX = GF([1 if np.random.uniform() < p else 0 for i in range(Hx.shape[1])])
    # eZ = GF([1 if np.random.uniform() < p else 0 for i in range(Hz.shape[1])])

    sigma_eX = Hx @ eX.T
    # sigma_eZ = Hz @ eZ.T
    e1 = ssf(sigma_eX, False)
    # e2 = ssf(sigma_eZ, True)
    print(e1)

{121, 161, 41, 81, 113, 25}
{89, 1}


In [328]:
e == set(np.where(eX)[0])

False

In [353]:
len(np.where(eX)[0])

13

In [395]:
%timeit ssf(sigma_eX, False)

54.3 ms ± 3.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
